<a href="https://colab.research.google.com/github/parwisenlared/MasterThesis/blob/master/ImageGeneration_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [170]:
!unzip -uq "/content/drive/My Drive/Colab Notebooks/dataset" -d "/content/drive/My Drive/Colab Notebooks/dataset"

In [168]:
import glob, os
#os.chdir("..")
print(os.getcwd())

#os.listdir(os.getcwd())
#os.chdir("drive/My Drive/Colab Notebooks/")
#print(os.getcwd())

/content/drive/My Drive/Colab Notebooks


In [ ]:
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

datagen = ImageDataGenerator(
        #rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        #shear_range=0.2,
        #zoom_range=0.2,
        #horizontal_flip=True,
        fill_mode='nearest')


for file in glob.glob("dataset/images/b&wZeroPressure/NeckerCubeDrawings/healthy/" + "*.png"):
    img = load_img(file)  # this is a PIL image
    x = img_to_array(img)  # this is a Numpy array with shape (1, 200, 200)
    x = x.reshape((1,) + x.shape)  # this is a Numpy array with shape (1, 200, 200)

# the .flow() command below generates batches of randomly transformed images
# and saves the results to the `preview/` directory
    i = 0
    for batch in datagen.flow(x, batch_size=1,
                          save_to_dir='dataset/images/b&wZeroPressure/NeckerCubeDrawings/healthy/', save_format='png'):
        i += 1
        if i > 23:
            break 
            
for file in glob.glob("dataset/images/b&wZeroPressure/NeckerCubeDrawings/PD/" + "*.png"):
    img = load_img(file)  # this is a PIL image
    x = img_to_array(img)  # this is a Numpy array with shape (1, 200, 200)
    x = x.reshape((1,) + x.shape)  # this is a Numpy array with shape (1, 200, 200)

# the .flow() command below generates batches of randomly transformed images
# and saves the results to the `preview/` directory
    i = 0
    for batch in datagen.flow(x, batch_size=1,
                          save_to_dir='dataset/images/b&wZeroPressure/NeckerCubeDrawings/PD/', save_format='png'):
        i += 1
        if i > 11:
            break 

In [164]:
import numpy as np
import os
from matplotlib import pyplot as plt
import cv2
import random
from PIL import Image
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img


DATADIR = "dataset/images/b&wZeroPressure/NeckerCubeDrawings"

# All the categories you want your neural network to detect
CATEGORIES = ["healthy", "PD"]

# The size of the images that your neural network will use

IMG_SIZE = 200

training_data = []

def create_training_data():
  for file in glob.glob("dataset/images/b&wZeroPressure/NeckerCubeDrawings/healthy/" + "*.png"):
      img = load_img(file, color_mode = "grayscale")  # this is a PIL image
      new_image = img_to_array(img)  # this is a Numpy array with shape (200, 200,1)
      training_data.append([new_image, 0])
  
  print(len(training_data))

  for file in glob.glob("dataset/images/b&wZeroPressure/NeckerCubeDrawings/PD/" + "*.png"):
      img = load_img(file, color_mode = "grayscale")  # this is a PIL image
      new_image = img_to_array(img)  # this is a Numpy array with shape (200, 200,1)
      training_data.append([new_image, 1]) # class is 1 


#X = np.array(X).reshape(-1, IMG_SIZE, IMG_SIZE, 1)
create_training_data()

random.shuffle(training_data) #shufles the data so it's not control and patient

X = [] #features
y = [] #labels
#y = np.array(y)

for features, label in training_data:
	X.append(features)
	y.append(label)

len(X), len(y)

1202


(2211, 2211)

## Define CNN model

In [152]:
def split_dataset(x,y):

    # train and test set (90% training, 10% test)
    X_train = X[:int(0.9*int(len(X)))]
    y_train = y[:int(0.9*int(len(X)))]
    X_test = X[int(0.9*int(len(X))):]
    y_test = y[int(0.9*int(len(y))):]
    
    # validation set (90% training, 10% validation)
    X_train = X_train[:int(0.9*int(len(X_train)))]
    y_train = y_train[:int(0.9*int(len(y_train)))]
    
    X_val = X_train[int(0.9*int(len(X_train))):]
    y_val = y_train[int(0.9*int(len(y_train))):]
    
    return X_train, y_train, X_test, y_test, X_val, y_val

#len(X_val), len(y_val)



In [165]:
# To check 
X_train, y_train, X_test, y_test, X_val, y_val = split_dataset(X,y)

X_train, y_train, X_test, y_test, X_val, y_val = prepare_dataset(X_train, y_train, X_test, y_test, X_val, y_val)

print(len(X_val), len(y_val),len(X_train), len(y_train), len(X_test), len(y_test))


ValueError: ignored

In [160]:
def prepare_dataset(X_train, y_train, X_test, y_test, X_val, y_val):

    # reshape training sets to prepare for CNN
    X_train = np.array(X_train).reshape(-1, IMG_SIZE, IMG_SIZE, 1)
    y_train = np.array(y_train)
    
    X_test = np.array(X_test).reshape(-1, IMG_SIZE, IMG_SIZE, 1)
    y_test = np.array(y_test)
    
    X_val = np.array(X_val).reshape(-1, IMG_SIZE, IMG_SIZE, 1)
    y_val = np.array(y_val)
    
    # one hot encoding of pixels
    y_train = to_categorical(y_train)
    y_test = to_categorical(y_test)
    y_val = to_categorical(y_val)
    
    train_norm = X_train.astype('float32')
    test_norm = X_test.astype('float32')
    val_norm = X_val.astype('float32')

    # normalize to range 0-1
    X_train = train_norm / 255.0
    X_test = test_norm / 255.0
    X_val = val_norm / 255.0
    # return dataset
    
    return X_train, y_train, X_test, y_test, X_val, y_val

In [163]:
f

or i in X_test:
  print(i.shape)

(200, 200, 1)
(200, 200, 1)
(150, 150, 1)
(200, 200, 1)
(200, 200, 1)
(200, 200, 1)
(150, 150, 1)
(200, 200, 1)
(150, 150, 1)
(150, 150, 1)
(200, 200, 1)
(200, 200, 1)
(150, 150, 1)
(200, 200, 1)
(150, 150, 1)
(200, 200, 1)
(150, 150, 1)
(200, 200, 1)
(200, 200, 1)
(150, 150, 1)
(200, 200, 1)
(200, 200, 1)
(200, 200, 1)
(200, 200, 1)
(150, 150, 1)
(150, 150, 1)
(200, 200, 1)
(150, 150, 1)
(150, 150, 1)
(200, 200, 1)
(200, 200, 1)
(200, 200, 1)
(150, 150, 1)
(200, 200, 1)
(200, 200, 1)
(150, 150, 1)
(200, 200, 1)
(200, 200, 1)
(200, 200, 1)
(200, 200, 1)
(150, 150, 1)
(200, 200, 1)
(200, 200, 1)
(150, 150, 1)
(200, 200, 1)
(200, 200, 1)
(200, 200, 1)
(200, 200, 1)
(150, 150, 1)
(200, 200, 1)
(200, 200, 1)
(150, 150, 1)
(200, 200, 1)
(150, 150, 1)
(200, 200, 1)
(200, 200, 1)
(200, 200, 1)
(150, 150, 1)
(150, 150, 1)
(200, 200, 1)
(150, 150, 1)
(200, 200, 1)
(200, 200, 1)
(200, 200, 1)
(200, 200, 1)
(150, 150, 1)
(150, 150, 1)
(200, 200, 1)
(150, 150, 1)
(200, 200, 1)
(150, 150, 1)
(200, 

In [153]:
# test harness for evaluating models on the cifar10 dataset
import sys
from matplotlib import pyplot
from keras.datasets import cifar10, mnist
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.optimizers import SGD
from sklearn.metrics import cohen_kappa_score

 
# define cnn model
def define_model():
  model = Sequential()
  model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', input_shape=(200, 200, 1)))
  model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
  model.add(MaxPooling2D((2, 2)))
  model.add(Dropout(0.2))
  model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
  model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
  model.add(MaxPooling2D((2, 2)))
  model.add(Dropout(0.3))
  model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
  model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
  model.add(MaxPooling2D((2, 2)))
  model.add(Dropout(0.4))
  model.add(Flatten())
  model.add(Dense(128, activation='relu', kernel_initializer='he_uniform'))
  model.add(Dropout(0.5))
  model.add(Dense(2, activation='softmax'))
  # compile model
  opt = SGD(lr=0.001, momentum=0.9)
  model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
  print("Model defined")
  return model
 
# plot diagnostic learning curves
def summarize_diagnostics(history):
	# plot loss
	pyplot.subplot(211)
	pyplot.title('Cross Entropy Loss')
	pyplot.plot(history.history['loss'], color='blue', label='train')
	pyplot.plot(history.history['val_loss'], color='orange', label='test')
	# plot accuracy
	pyplot.subplot(212)
	pyplot.title('Classification Accuracy')
	pyplot.plot(history.history['accuracy'], color='blue', label='train')
	pyplot.plot(history.history['val_accuracy'], color='orange', label='test')
    #pyplot.show()
	# save plot to file
	filename = sys.argv[0].split('/')[-1]
	pyplot.savefig(filename + '_plot.png')
	pyplot.close()
    
 
# run the test harness for evaluating a model
def run_test_harness(x,y):
    
    #X_train, y_train, X_test, y_test = create_test(X,y)
    
    #X_train, y_train, X_val, y_val = create_validation(X_train,y_train)
    
    X_train, y_train, X_test, y_test, X_val, y_val = split_dataset(X,y)
    
    X_train, y_train, X_test, Y_test, X_val, y_val = prepare_dataset(X_train, y_train, X_test, y_test, X_val, y_val)
    
    print(len(X_val), len(y_val))
          
    model = define_model()
    # fit model
    history = model.fit(X_train, y_train, epochs=10, batch_size=24, validation_data=(X_val, y_val), verbose=2)
    # evaluate model
    _, acc = model.evaluate(X_test, Y_test, verbose=1)
    print('Test Accuracy: %.3f' % (acc * 100.0))

    # Kappa statistic

    model.predict(y_test)
    yhat_classes = model.predict_classes(testX, verbose=0)
    y_values = y_test

    kappa = cohen_kappa_score(y_values, yhat_classes)
    kappa, y_values

    cohen_kappa_score(y_test, predictions)
    # learning curves
    summarize_diagnostics(history)

In [ ]:
run_test_harness(X,y)

In [158]:
    model = define_model()

    X_train, y_train, X_test, y_test, X_val, y_val = split_dataset(X,y)
    X_train, y_train, X_test, Y_test, X_val, y_val = prepare_dataset(X_train, y_train, X_test, y_test, X_val, y_val)


    yhat_classes = model.predict_classes(X_test, verbose=1)
    #yhat_classes = yhat_classes[:,0]
    yhat_classes 
    y_values = y_test

    kappa = cohen_kappa_score(y_values, yhat_classes)
    kappa, y_values

Model defined


ValueError: ignored